In [5]:
using CairoMakie, QuantEcon, LinearAlgebra, Optim, Dierckx

In [6]:
include("crra_code.jl")
include("ces_code.jl")

hpi_CES (generic function with 1 method)

In [7]:
function invariant_distribution(M, O, X, Y, Inv, policy, Π, Avals, n, pars)
    (; na, nz) = pars
    for i in 1:na
        for j in 1:nz
            if policy[i,j] <+ Avals[1]
                M[i,j,1,:] = Π[j,:]
            elseif policy[i,j] >= Avals[end]
                M[i,j,end,:] = Π[j,:]
            else
                index = findfirst(x -> x > policy[i,j], Avals)
                π = (Avals[index] - policy[i,j]) / (Avals[index] - Avals[index-1])
                M[i,j,index-1,:] = π * Π[j,:]
                M[i,j,index,:] = (1-π) * Π[j,:]
            end
        end
    end
    O = reshape(M, n, n)
    for i in 1:na
        O[i,:] = O[i,:] / sum(O[i,:])
    end
    X = reduce(vcat, [I(n) - O', ones(n)])
    Y = [zeros(n);1]
    # Inv = lstsq(X, Y).solution
    Inv = X \ Y
    Inv[Inv .< 0] .= 0
    Inv = Inv / sum(Inv)
    return Inv
end

invariant_distribution (generic function with 1 method)

In [8]:
function labour_supply(Π, Zvals, pars)
    (; nz) = pars
    L = zeros(nz)
    L2 = reduce(vcat, [I(nz) - Π', ones(nz)])
    L3 = [zeros(nz);1]
    L = L2 \ L3
    L[L .< 0] .= 0
    L = L / sum(L)
    return L
end

labour_supply (generic function with 1 method)

In [ ]:
function equilibrium(pars)
    (; na, nz, toler_prices, maxiter_prices, r, w, β, A, α) = pars
    n = na * nz
    M = zeros(na, nz, na, nz)
    O = zeros(n, n)
    X = zeros(n+1,n)
    Y = zeros(n+1)
    Inv = zeros(n)
    Π, Zvals = ar1(pars)
    Avals = exp_grid(pars)
    v_init = zeros(na,nz)
    v_new = similar(v_init)
    policy = similar(v_init)
    initial_distribution = zeros(n,1)
    L = labour_supply(Π, Zvals, pars)
    Ks, Kd = 1, 1
    iter = 0
    error = toler + 1
    rnew = 1/β - 1
    r_iter = (r + rnew) / 2
    if iter == 0
        println("Iterating on prices...")
    end
    while ((error > toler_prices) && (iter < maxiter_prices))
        println("//////////////////////")
        println("Price Iteration: $iter")
        Kd = (A^α * L ^ (1-α)) / (r_iter + δ)^(1/(1-α))
        w_iter = (1-α) * A * (Kd/L)^α
        pars = (; r = r_iter, w = w_iter, ...)
        v_new, policy = hpi(v_init, policy, Π, Zvals, Avals, v_new, pars)
        println("Converged for r = $r_iter, w = $w_iter")
        Invariant = invariant_distribution(M, O, X, Y, Inv, policy, Π, Avals, n, pars)
        G = reshape(policy, n, 1)
